In [1]:
import pandas as pd
import numpy as np

In [2]:
from filters import Filters
from frequency import Frequency
BASE_DIR = "newdata/data_excel_finalmatrix_modified/"

In [3]:
ALL_FILTERS = Filters(BASE_DIR)
FREQ = Frequency()

In [4]:
import os
TARGET_DIR = "code_32/"

In [9]:
if not os.path.exists(TARGET_DIR):
    os.mkdir(TARGET_DIR)
if not os.path.exists(TARGET_DIR+"original"):
    os.mkdir(TARGET_DIR+"original")
if not os.path.exists(TARGET_DIR+"first_der"):
    os.mkdir(TARGET_DIR+"first_der")
if not os.path.exists(TARGET_DIR+"colored_plots"):
    os.mkdir(TARGET_DIR+"colored_plots")
if not os.path.exists(TARGET_DIR+"segment_plots"):
    os.mkdir(TARGET_DIR+"segment_plots")
if not os.path.exists(TARGET_DIR+"segment_plots_new"):
    os.mkdir(TARGET_DIR+"segment_plots_new")
if not os.path.exists(TARGET_DIR+"freq_segment"):
    os.mkdir(TARGET_DIR+"freq_segment")
if not os.path.exists(TARGET_DIR+"freq_segment/data"):
    os.mkdir(TARGET_DIR+"freq_segment/data")
if not os.path.exists(TARGET_DIR+"freq_segment/plots"):
    os.mkdir(TARGET_DIR+"freq_segment/plots")
if not os.path.exists(TARGET_DIR+"segment_plots_morph"):
    os.mkdir(TARGET_DIR+"segment_plots_morph")
if not os.path.exists(TARGET_DIR+"freq_segment/morph"):
    os.mkdir(TARGET_DIR+"freq_segment/morph")
if not os.path.exists(TARGET_DIR+"freq_segment/morph/data"):
    os.mkdir(TARGET_DIR+"freq_segment/morph/data")
if not os.path.exists(TARGET_DIR+"freq_segment/morph/plots"):
    os.mkdir(TARGET_DIR+"freq_segment/morph/plots")

In [12]:
for file in os.listdir(BASE_DIR):
    if "32" in file:
        data = ALL_FILTERS.read_and_preprocess_data(file)
        num_lines = np.max(data[:,0])
        data = ALL_FILTERS.conservative_smoothing(data,5)
        data = ALL_FILTERS.apply_filter(data,ALL_FILTERS.GAUSSIAN_KERNEL,5)
        data = ALL_FILTERS.opening(data,ALL_FILTERS.MORPHOLOGICAL_FILTER)
        #ALL_FILTERS.plot_data(data,TARGET_DIR+"original/"+file[:-5]+".png")
        first_der = ALL_FILTERS.apply_filter(data,ALL_FILTERS.FIRST_DERIVATIVE,5)
        #thresh_less = ALL_FILTERS.apply_range_thresh(first_der, int(num_lines*1./3.),int(num_lines*2./3.))
        #thresh_more = ALL_FILTERS.apply_range_thresh(first_der, int(num_lines*2./3.),int(num_lines))
        #ALL_FILTERS.plot_data(first_der,TARGET_DIR+"first_der/"+file[:-5]+".png",isDerivative=True)
        #ALL_FILTERS.plot_data(thresh_less,TARGET_DIR+"first_der/"+file[:-5]+"_less.png",isDerivative=True)
        #ALL_FILTERS.plot_data(thresh_more,TARGET_DIR+"first_der/"+file[:-5]+"_more.png",isDerivative=True)
        #data_blue_less, data_red_less = ALL_FILTERS.seperate_data_range(data, first_der, \
        #                                            int(num_lines*1./3.),int(num_lines*2./3.))
        #data_blue_more, data_red_more = ALL_FILTERS.seperate_data_range(data, first_der, \
        #                                            int(num_lines*2./3.),int(num_lines))
        #ALL_FILTERS.plot_colored_data(data_blue_less, data_red_less,TARGET_DIR+\
        #                              "colored_plots/"+file[:-5]+"_less.png")
        #ALL_FILTERS.plot_colored_data(data_blue_more, data_red_more,TARGET_DIR+\
        #                              "colored_plots/"+file[:-5]+"_more.png")
        data_blue_med, data_red_med = ALL_FILTERS.seperate_data_range(data, first_der, \
                                                    int(num_lines*.4),int(num_lines*.7))
        #ALL_FILTERS.plot_colored_data(data_blue_med, data_red_med,TARGET_DIR+\
        #                              "colored_plots/"+file[:-5]+"_med.png")
        segments = ALL_FILTERS.extract_segments(data,first_der,int(num_lines*.4),int(num_lines*.7))
        ALL_FILTERS.plot_colored_data_with_segments(data_blue_med,data_red_med,segments,\
                                                   TARGET_DIR+"segment_plots_morph/"+file[:-5]+"_morph.png")
        seperated_segs = ALL_FILTERS.seperate_segments(data,segments)
        #print(len(seperated_segs))
        sheet = 0
        writer = pd.ExcelWriter(TARGET_DIR+"freq_segment/morph/data/"+file[:-5]+".xlsx",engine='xlsxwriter')
        for seg in seperated_segs:
            cts = FREQ.getFrequencies(seg,num_lines)
            df = pd.DataFrame(cts)
            sheet += 1
            df.to_excel(writer,sheet_name='Sheet{}'.format(sheet),header=["Line Number","Frequency"],index=False)
        writer.save()

/home/ayush/anaconda3/lib/python3.6/site-packages/matplotlib/pyplot.py:523: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [7]:
for file in os.listdir(TARGET_DIR+"freq_segment/morph/data"):
    all_sheets = pd.read_excel(TARGET_DIR+"freq_segment/morph/data/"+file,sheetname=None)
    for name,sheet in all_sheets.items():
        data = np.array(sheet)
        segnum = name[5:]
        FREQ.plot_freq(data,saveplace=TARGET_DIR+"freq_segment/morph/plots/"+file[:-5]+"_"+segnum+".png")